In [1]:
import cx_Oracle
import pandas as pd
import datetime
import time
import re

# cx_Oracle.init_oracle_client(lib_dir="C://Users//User//Downloads//instantclient-basic-windows.x64-21.10.0.0.0dbru//instantclient_21_10")


In [3]:
pip install cx_Oracle --upgrade

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

def format_value(value):
    if value is None:
        return 'NULL'
    elif isinstance(value, str):
        return f"'{value}'"
    elif isinstance(value, datetime.date):
        formatted_date = value.strftime("%Y-%m-%d %H:%M:%S")
        return f"TO_DATE('{formatted_date}', 'YYYY-MM-DD HH24:MI:SS')"
    else:
        return str(value)

In [7]:
def delete_def_script(target_table,p_map_code):
    delete_query = f"DELETE FROM {target_table} WHERE MAP_CODE = '{p_map_code}'"
    # Append a slash on the next line
    delete_query += ";\n"
    print(delete_query)
    # Write the first query to a select.sql file
    with open('Script.sql', 'a') as sql_file:
        sql_file.write(delete_query)
        sql_file.write('/\n')

In [3]:
def delete_vm_script(target_vm_table,target_table,p_map_code):
#     delete_vm_query = f"""DELETE FROM {target_vm_table} WHERE MAP_DEFINITION_ID = {target_existing_map_def_id}"""
    delete_vm_query = f"""DELETE {target_vm_table} 
                          Where map_definition_id = (SELECT
                                 map_definition_id
                            FROM
                                {target_table}
                            WHERE MAP_CODE = '{p_map_code}')"""
    delete_vm_query += ";\n"
    print(delete_vm_query)
    with open('Script.sql', 'w') as sql_file:
        sql_file.write(delete_vm_query)
        sql_file.write('/\n')

In [4]:
def vm_copy_with_map_code(p_map_code, OVERRIDE_FLAG=False):
    # Source database connection
    source_conn = cx_Oracle.connect(
        user="apps",
        password="apps",
        dsn="omsddb.cswg.com:1521/omsd_app"
    )

    # Target database connection
    target_conn = cx_Oracle.connect(
        user="apps",
        password="apps",
        dsn="omsddb.cswg.com:1521/omsd_app"
    )

    # Create cursors for both connections
    source_cursor = source_conn.cursor()
    target_cursor = target_conn.cursor()

    # Select data from the source table
    def_query = f"""SELECT * FROM APPS.XXCMN_VALUE_MAP_DEF WHERE MAP_CODE = '{p_map_code}'"""
    source_cursor.execute(def_query)

    # Fetch the data and the column names
    def_df = source_cursor.fetchall()

    # Retrieve the column names from the cursor description
    def_column_names = [desc[0] for desc in source_cursor.description]

    # Create a DataFrame and set the column names
    def_df = pd.DataFrame(def_df, columns=def_column_names)
#     print(def_df)

    # Now you can reference columns by their names
    pn_map_def_id = int(def_df['MAP_DEFINITION_ID'].iloc[0])
    print('pn_map_def_id :',pn_map_def_id)

    vm_query = f"""SELECT *
                    FROM APPS.XXCMN_VALUE_MAP 
                    WHERE MAP_DEFINITION_ID = {pn_map_def_id}
                    ORDER BY map_id"""
    source_cursor.execute(vm_query)

    # Fetch the data and the column names
    vm_df = source_cursor.fetchall()

    # Retrieve the column names from the cursor description
    vm_column_names = [desc[0] for desc in source_cursor.description]

    # Create a DataFrame and set the column names
    vm_df = pd.DataFrame(vm_df, columns=vm_column_names)
#     print(vm_df)
    # column_types = vm_df.dtypes
    # for i in column_types:
    # # Print the data types
    #     print('Print the data types',i)

    # Close the source cursor and connection
    source_cursor.close()
    source_conn.close()

    # Construct the INSERT INTO statement for DEF

    def_df = def_df.iloc[:, 1:]

    ########################################################################
    ##################### Specify the target table name ####################
    ########################################################################

    target_table = "OMS_CMN_DATA.XXCMN_VALUE_MAP_DEF_TEST"
    target_vm_table = "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST"
    target_seq = "OMS_CMN_DATA.XXCMN_VALUE_MAP_DEF_TEST_S.NEXTVAL"
    target_vm_seq = "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S.NEXTVAL"

    # Construct the INSERT INTO statement
    insert_def = f"""
        INSERT INTO {target_table} (
            MAP_DEFINITION_ID, {', '.join(def_column_names[1:])}  
        ) VALUES (
            :MAP_DEFINITION_ID ,:{', :'.join(def_column_names[1:])} 
        )
    """
    vm_df = vm_df.iloc[:, 2:]

    # Construct the INSERT INTO statement
    vm_column_names = vm_df.columns.tolist()
    print('vm_column_names',vm_column_names)

    insert_vm = f"""
        INSERT INTO {target_vm_table} (
            MAP_ID, MAP_DEFINITION_ID, {', '.join(vm_column_names)}  
        ) VALUES (
            :MAP_ID, :MAP_DEFINITION_ID, :{', :'.join(vm_column_names)} 
        )
    """
#     print(insert_vm)
    
    print('pn_map_def_id :',pn_map_def_id)
#     target_def_seq = get_next_sequence_value(target_cursor,target_seq)
#     print(target_def_seq,type(target_def_seq))
#     values = {'MAP_DEFINITION_ID': target_def_seq}
    target_cursor.execute(f"SELECT {target_seq} FROM dual")
    next_val = target_cursor.fetchone()[0]
    values = {'MAP_DEFINITION_ID': next_val} 
    # Populate the values dictionary from the DataFrame
    for col in def_column_names[1:]:
        values[col] = def_df[col].iloc[0]  # Get values from the DataFrame
    
    
    # Execute the INSERT statement with the values dictionary
    try:
        qry_map_code = f"SELECT MAP_DEFINITION_ID,MAP_CODE FROM {target_table} WHERE MAP_CODE = '{p_map_code}'"
        target_cursor.execute(qry_map_code)
        data_exists = target_cursor.fetchone()
        if data_exists:
            target_existing_map_def_id =data_exists[0]
            print(target_existing_map_def_id)
            existing_map_code = data_exists[1]
             
        if OVERRIDE_FLAG:
            # If override flag is true, delete and insert
            print('pn_map_def_id : ',pn_map_def_id)
            delete_vm_script(target_vm_table,target_table,p_map_code)
            delete_def_script(target_table,p_map_code)
            insert_def_script(insert_def,values)
            insert_vm_script(insert_vm,vm_df,target_vm_seq,next_val,vm_column_names)
            with open('Script.sql', 'a') as sql_file:
                    sql_file.write('\n')
                    sql_file.write('COMMIT;')
                    sql_file.write('\n/')
        else:
            # If override flag is false, check if data exists
            if data_exists:
                # Data exists, so skip
                print('As Map Code Value already Exists and You Dont want to Override the Existing Value Skipped')
                pass
            else:
                # Data doesn't exist, so insert
                print('Data doesn\'t exist, so insert script is Prepared in .sql file')
                with open('Script.sql', 'w') as sql_file:
                    sql_file.write('\n')
                insert_def_script(insert_def,values)
                insert_vm_script(insert_vm,vm_df,target_vm_seq,next_val,vm_column_names)
                with open('Script.sql', 'a') as sql_file:
                    sql_file.write('\n')
                    sql_file.write('COMMIT;')
                    sql_file.write('\n/')
    except cx_Oracle.Error as error:
        print(f"Error executing DEF INSERT statement: {error}")
    


    # Close the target cursor and connection
    target_cursor.close()
    target_conn.close()


In [5]:
def insert_def_script(insert_def,values):
    insert_script = insert_def.strip().replace(':MAP_DEFINITION_ID', str(values['MAP_DEFINITION_ID']))
#     print('values in def :values',values)
    for col in values:
#         print('col',col)
        if col != 'MAP_DEFINITION_ID':
#             print(values[col])
            value = values[col]
            formatted_value = format_value(value)
            insert_script = insert_script.replace(f':{col}', formatted_value)
    # Print the INSERT script with actual values
#     print("Insert Query with Actual Values:")
    insert_script += ";\n"
#     print(insert_script)
    with open('Script.sql', 'a') as sql_file:
        sql_file.write(insert_script)
        sql_file.write('/\n')

In [9]:
# def insert_vm_script(insert_vm,vm_df,target_vm_seq,next_val,vm_column_names): 
#     print(insert_vm)
#     print(target_vm_seq)
#     print("Inside insert_vm_script:")
#     insert_script = insert_vm.strip()
#     for index, row in vm_df.iterrows():
#         # Get the next sequence value
#         vm_values = {'MAP_ID': target_vm_seq}
#         vm_values['MAP_DEFINITION_ID'] = next_val  # Use the provided map_definition_id

#         # Populate the values dictionary from the DataFrame row
#         for col in vm_column_names:
#             vm_values[col] = row[col]
#             formatted_value = format_value(vm_values)
#             insert_script = insert_script.replace(f':{col}', formatted_value)
#         print(insert_script)

def insert_vm_script(insert_vm, vm_df, target_vm_seq, next_val, vm_column_names):
#     print('insert_vm:\n',insert_vm)
#     print(target_vm_seq)
#     print(f"Inside insert_vm_script:\n")
#     print(insert_vm)
    for index, row in vm_df.iterrows():
        # Get a copy of the original insert_vm script for each row
        vm_values = {'MAP_ID': target_vm_seq}
        vm_values['MAP_DEFINITION_ID'] = next_val  # Use the provided map_definition_id
        # Replace specific columns with desired values
        vm_values['CREATED_BY'] = -1
        vm_values['LAST_UPDATED_BY'] = -1
        vm_values['CREATION_DATE'] = 'SYSDATE'
        vm_values['LAST_UPDATE_DATE'] = 'SYSDATE'
#         insert_script = insert_vm.strip().replace(':MAP_ID', str(vm_values['MAP_ID']))
#         insert_script = insert_vm.strip().replace(':MAP_DEFINITION_ID', str(vm_values['MAP_DEFINITION_ID']))
        insert_script = insert_vm.replace(':MAP_ID', str(vm_values['MAP_ID'])).replace(':MAP_DEFINITION_ID', str(vm_values['MAP_DEFINITION_ID']))
#         print('insert_script after updating mapid and def \n: ',insert_script)
        # Populate the values dictionary from the DataFrame row
        for col in vm_column_names:
#             print('vm_column_names')
#             print(vm_column_names)
            if col not in ('CREATED_BY', 'LAST_UPDATED_BY', 'CREATION_DATE', 'LAST_UPDATE_DATE'):
            vm_values[col] = vm_df[col].iloc[index]
            value=vm_values[col]
            formatted_value = format_value(value)
#             print("m_values[col],'  ',col,' ',formatted_value")
#             print(vm_values[col],'  ',col,' ',formatted_value)
#             print(insert_script)
#             insert_script = insert_script.replace(f':{col}', formatted_value)
            # Use regular expressions to match the exact placeholder and replace it
            placeholder = f':{col}(?=\s|,|$)'  # Match ":col " or ":col," or ":col" at the end of the string
            insert_script = re.sub(placeholder, formatted_value, insert_script, count=1)
        
        with open('Script.sql', 'a') as sql_file:
            insert_script = insert_script.rstrip()
            insert_script += ";\n"
            sql_file.write(insert_script)
            sql_file.write('/\n')
            
#         print('------------------vm_values[col]----------------')
#         print(vm_values[col])
#         print(f'------------------insert_script{index}----------------')
        print(insert_script)
            
        
#         for col in vm_column_names:
#             print('col',col)
#             value = row[col]
#             formatted_value = 'NULL' if value is None else format_value(value)
#             insert_script = insert_script.replace(f':{col}', formatted_value)
# #             vm_values[col] = row[col]
# #             formatted_value = format_value(vm_values[col])
# #             formatted_value = 'NULL' if value is None else format_value(value)
# #             insert_script = insert_script.replace(f':{col}', formatted_value)
        
#         # Now, for each row, you'll have a separate insert statement
#         print(insert_script)

        
 

In [10]:
%%time
OVERRIDE_FLAG = True
p_map_code = "OMS_API_ENDPOINT"
# p_map_code = "YOGI"
vm_copy_with_map_code(p_map_code, OVERRIDE_FLAG)

pn_map_def_id : 2601
vm_column_names ['N_VALUE1', 'C_VALUE1', 'D_VALUE1', 'N_VALUE2', 'C_VALUE2', 'D_VALUE2', 'N_VALUE3', 'C_VALUE3', 'D_VALUE3', 'N_VALUE4', 'C_VALUE4', 'D_VALUE4', 'N_VALUE5', 'C_VALUE5', 'D_VALUE5', 'N_VALUE6', 'C_VALUE6', 'D_VALUE6', 'N_VALUE7', 'C_VALUE7', 'D_VALUE7', 'N_VALUE8', 'C_VALUE8', 'D_VALUE8', 'N_VALUE9', 'C_VALUE9', 'D_VALUE9', 'N_VALUE10', 'C_VALUE10', 'D_VALUE10', 'N_VALUE11', 'C_VALUE11', 'D_VALUE11', 'N_VALUE12', 'C_VALUE12', 'D_VALUE12', 'N_VALUE13', 'C_VALUE13', 'D_VALUE13', 'N_VALUE14', 'C_VALUE14', 'D_VALUE14', 'N_VALUE15', 'C_VALUE15', 'D_VALUE15', 'N_VALUE16', 'C_VALUE16', 'D_VALUE16', 'N_VALUE17', 'C_VALUE17', 'D_VALUE17', 'N_VALUE18', 'C_VALUE18', 'D_VALUE18', 'N_VALUE19', 'C_VALUE19', 'D_VALUE19', 'N_VALUE20', 'C_VALUE20', 'D_VALUE20', 'ACTIVE_BEGIN_DATE', 'ACTIVE_END_DATE', 'CREATED_BY', 'CREATION_DATE', 'LAST_UPDATED_BY', 'LAST_UPDATE_DATE', 'LAST_UPDATE_LOGIN']
pn_map_def_id : 2601
362
pn_map_def_id :  2601
DELETE OMS_CMN_DATA.XXCMN_VALUE


        INSERT INTO OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST (
            MAP_ID, MAP_DEFINITION_ID, N_VALUE1, C_VALUE1, D_VALUE1, N_VALUE2, C_VALUE2, D_VALUE2, N_VALUE3, C_VALUE3, D_VALUE3, N_VALUE4, C_VALUE4, D_VALUE4, N_VALUE5, C_VALUE5, D_VALUE5, N_VALUE6, C_VALUE6, D_VALUE6, N_VALUE7, C_VALUE7, D_VALUE7, N_VALUE8, C_VALUE8, D_VALUE8, N_VALUE9, C_VALUE9, D_VALUE9, N_VALUE10, C_VALUE10, D_VALUE10, N_VALUE11, C_VALUE11, D_VALUE11, N_VALUE12, C_VALUE12, D_VALUE12, N_VALUE13, C_VALUE13, D_VALUE13, N_VALUE14, C_VALUE14, D_VALUE14, N_VALUE15, C_VALUE15, D_VALUE15, N_VALUE16, C_VALUE16, D_VALUE16, N_VALUE17, C_VALUE17, D_VALUE17, N_VALUE18, C_VALUE18, D_VALUE18, N_VALUE19, C_VALUE19, D_VALUE19, N_VALUE20, C_VALUE20, D_VALUE20, ACTIVE_BEGIN_DATE, ACTIVE_END_DATE, CREATED_BY, CREATION_DATE, LAST_UPDATED_BY, LAST_UPDATE_DATE, LAST_UPDATE_LOGIN  
        ) VALUES (
            OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S.NEXTVAL, 502, nan, 'TMS Move Stops GS', NULL, NULL, 'TMS_MOVE_STOPS_GS', NULL, NULL, '

In [61]:
# def insert_vm_script(insert_vm, vm_values, target_vm_seq):
print(insert_vm)

NameError: name 'insert_vm' is not defined

In [67]:
import datetime

def format_value(value):
    if value is None:
        return 'NULL'
    elif isinstance(value, str):
        return f"'{value}'"
    elif isinstance(value, datetime.date):
        formatted_date = value.strftime("%Y-%m-%d %H:%M:%S")
        return f"TO_DATE('{formatted_date}', 'YYYY-MM-DD HH24:MI:SS')"
    else:
        return str(value)

def build_insert_script(vm_values, insert_vm, target_vm_seq):
    for values in vm_values:
        print(values)

    return insert_statement


# Sample vm_values, insert_vm, and target_vm_seq
vm_values = {'MAP_ID': 'OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S.NEXTVAL', 'MAP_DEFINITION_ID': 296, 'N_VALUE1': None, 'C_VALUE1': 'AWI', 'D_VALUE1': None, 'N_VALUE2': None, 'C_VALUE2': 'Immediate', 'D_VALUE2': None, 'N_VALUE3': None, 'C_VALUE3': 'Customer Pickup', 'D_VALUE3': None, 'N_VALUE4': None, 'C_VALUE4': '20', 'D_VALUE4': None, 'N_VALUE5': None, 'C_VALUE5': 'Y', 'D_VALUE5': None, 'N_VALUE6': None, 'C_VALUE6': 'N', 'D_VALUE6': None, 'N_VALUE7': None, 'C_VALUE7': None, 'D_VALUE7': None, 'N_VALUE8': None, 'C_VALUE8': None, 'D_VALUE8': None, 'N_VALUE9': None, 'C_VALUE9': None, 'D_VALUE9': None, 'N_VALUE10': None, 'C_VALUE10': None, 'D_VALUE10': None, 'N_VALUE11': None, 'C_VALUE11': None, 'D_VALUE11': None, 'N_VALUE12': None, 'C_VALUE12': None, 'D_VALUE12': None, 'N_VALUE13': None, 'C_VALUE13': None, 'D_VALUE13': None, 'N_VALUE14': None, 'C_VALUE14': None, 'D_VALUE14': None, 'N_VALUE15': None, 'C_VALUE15': None, 'D_VALUE15': None, 'N_VALUE16': None, 'C_VALUE16': None, 'D_VALUE16': None, 'N_VALUE17': None, 'C_VALUE17': None, 'D_VALUE17': None, 'N_VALUE18': None, 'C_VALUE18': None, 'D_VALUE18': None, 'N_VALUE19': None, 'C_VALUE19': None, 'D_VALUE19': None, 'N_VALUE20': None, 'C_VALUE20': None, 'D_VALUE20': None, 'ACTIVE_BEGIN_DATE': None, 'ACTIVE_END_DATE': None, 'CREATED_BY': '1088.0', 'CREATION_DATE': Timestamp('2021-08-03 09:46:28'), 'LAST_UPDATED_BY': '1088.0', 'LAST_UPDATE_DATE': Timestamp('2022-06-03 10:30:58'), 'LAST_UPDATE_LOGIN': None}
{'MAP_ID': 'OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S.NEXTVAL', 'MAP_DEFINITION_ID': 296, 'N_VALUE1': None, 'C_VALUE1': 'AWI', 'D_VALUE1': None, 'N_VALUE2': None, 'C_VALUE2': 'None', 'D_VALUE2': None, 'N_VALUE3': None, 'C_VALUE3': 'Truck', 'D_VALUE3': None, 'N_VALUE4': None, 'C_VALUE4': '10', 'D_VALUE4': None, 'N_VALUE5': None, 'C_VALUE5': 'N', 'D_VALUE5': None, 'N_VALUE6': None, 'C_VALUE6': 'N', 'D_VALUE6': None, 'N_VALUE7': None, 'C_VALUE7': None, 'D_VALUE7': None, 'N_VALUE8': None, 'C_VALUE8': None, 'D_VALUE8': None, 'N_VALUE9': None, 'C_VALUE9': None, 'D_VALUE9': None, 'N_VALUE10': None, 'C_VALUE10': None, 'D_VALUE10': None, 'N_VALUE11': None, 'C_VALUE11': None, 'D_VALUE11': None, 'N_VALUE12': None, 'C_VALUE12': None, 'D_VALUE12': None, 'N_VALUE13': None, 'C_VALUE13': None, 'D_VALUE13': None, 'N_VALUE14': None, 'C_VALUE14': None, 'D_VALUE14': None, 'N_VALUE15': None, 'C_VALUE15': None, 'D_VALUE15': None, 'N_VALUE16': None, 'C_VALUE16': None, 'D_VALUE16': None, 'N_VALUE17': None, 'C_VALUE17': None, 'D_VALUE17': None, 'N_VALUE18': None, 'C_VALUE18': None, 'D_VALUE18': None, 'N_VALUE19': None, 'C_VALUE19': None, 'D_VALUE19': None, 'N_VALUE20': None, 'C_VALUE20': None, 'D_VALUE20': None, 'ACTIVE_BEGIN_DATE': None, 'ACTIVE_END_DATE': None, 'CREATED_BY': '1088.0', 'CREATION_DATE': Timestamp('2021-08-03 09:48:20'), 'LAST_UPDATED_BY': '1088.0', 'LAST_UPDATE_DATE': Timestamp('2021-08-03 09:48:20'), 'LAST_UPDATE_LOGIN': None}
{'MAP_ID': 'OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S.NEXTVAL', 'MAP_DEFINITION_ID': 296, 'N_VALUE1': None, 'C_VALUE1': 'GS', 'D_VALUE1': None, 'N_VALUE2': None, 'C_VALUE2': 'Immediate', 'D_VALUE2': None, 'N_VALUE3': None, 'C_VALUE3': 'Customer Pickup', 'D_VALUE3': None, 'N_VALUE4': None, 'C_VALUE4': '20', 'D_VALUE4': None, 'N_VALUE5': None, 'C_VALUE5': 'Y', 'D_VALUE5': None, 'N_VALUE6': None, 'C_VALUE6': 'N', 'D_VALUE6': None, 'N_VALUE7': None, 'C_VALUE7': None, 'D_VALUE7': None, 'N_VALUE8': None, 'C_VALUE8': None, 'D_VALUE8': None, 'N_VALUE9': None, 'C_VALUE9': None, 'D_VALUE9': None, 'N_VALUE10': None, 'C_VALUE10': None, 'D_VALUE10': None, 'N_VALUE11': None, 'C_VALUE11': None, 'D_VALUE11': None, 'N_VALUE12': None, 'C_VALUE12': None, 'D_VALUE12': None, 'N_VALUE13': None, 'C_VALUE13': None, 'D_VALUE13': None, 'N_VALUE14': None, 'C_VALUE14': None, 'D_VALUE14': None, 'N_VALUE15': None, 'C_VALUE15': None, 'D_VALUE15': None, 'N_VALUE16': None, 'C_VALUE16': None, 'D_VALUE16': None, 'N_VALUE17': None, 'C_VALUE17': None, 'D_VALUE17': None, 'N_VALUE18': None, 'C_VALUE18': None, 'D_VALUE18': None, 'N_VALUE19': None, 'C_VALUE19': None, 'D_VALUE19': None, 'N_VALUE20': None, 'C_VALUE20': None, 'D_VALUE20': None, 'ACTIVE_BEGIN_DATE': None, 'ACTIVE_END_DATE': None, 'CREATED_BY': '1088.0', 'CREATION_DATE': Timestamp('2023-08-23 09:24:52'), 'LAST_UPDATED_BY': '1088.0', 'LAST_UPDATE_DATE': Timestamp('2023-08-23 09:24:52'), 'LAST_UPDATE_LOGIN': None}
{'MAP_ID': 'OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S.NEXTVAL', 'MAP_DEFINITION_ID': 296, 'N_VALUE1': None, 'C_VALUE1': 'GS', 'D_VALUE1': None, 'N_VALUE2': None, 'C_VALUE2': 'None', 'D_VALUE2': None, 'N_VALUE3': None, 'C_VALUE3': 'Truck', 'D_VALUE3': None, 'N_VALUE4': None, 'C_VALUE4': '10', 'D_VALUE4': None, 'N_VALUE5': None, 'C_VALUE5': 'N', 'D_VALUE5': None, 'N_VALUE6': None, 'C_VALUE6': 'N', 'D_VALUE6': None, 'N_VALUE7': None, 'C_VALUE7': None, 'D_VALUE7': None, 'N_VALUE8': None, 'C_VALUE8': None, 'D_VALUE8': None, 'N_VALUE9': None, 'C_VALUE9': None, 'D_VALUE9': None, 'N_VALUE10': None, 'C_VALUE10': None, 'D_VALUE10': None, 'N_VALUE11': None, 'C_VALUE11': None, 'D_VALUE11': None, 'N_VALUE12': None, 'C_VALUE12': None, 'D_VALUE12': None, 'N_VALUE13': None, 'C_VALUE13': None, 'D_VALUE13': None, 'N_VALUE14': None, 'C_VALUE14': None, 'D_VALUE14': None, 'N_VALUE15': None, 'C_VALUE15': None, 'D_VALUE15': None, 'N_VALUE16': None, 'C_VALUE16': None, 'D_VALUE16': None, 'N_VALUE17': None, 'C_VALUE17': None, 'D_VALUE17': None, 'N_VALUE18': None, 'C_VALUE18': None, 'D_VALUE18': None, 'N_VALUE19': None, 'C_VALUE19': None, 'D_VALUE19': None, 'N_VALUE20': None, 'C_VALUE20': None, 'D_VALUE20': None, 'ACTIVE_BEGIN_DATE': None, 'ACTIVE_END_DATE': None, 'CREATED_BY': '1088.0', 'CREATION_DATE': Timestamp('2023-08-23 09:24:52'), 'LAST_UPDATED_BY': '1088.0', 'LAST_UPDATE_DATE': Timestamp('2023-08-23 09:24:52'), 'LAST_UPDATE_LOGIN': None}
insert_vm = """INSERT INTO OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST (
            MAP_ID, MAP_DEFINITION_ID, N_VALUE1, C_VALUE1, D_VALUE1, N_VALUE2, C_VALUE2, D_VALUE2, N_VALUE3, C_VALUE3, D_VALUE3, N_VALUE4, C_VALUE4, D_VALUE4, N_VALUE5, C_VALUE5, D_VALUE5, N_VALUE6, C_VALUE6, D_VALUE6, N_VALUE7, C_VALUE7, D_VALUE7, N_VALUE8, C_VALUE8, D_VALUE8, N_VALUE9, C_VALUE9, D_VALUE9, N_VALUE10, C_VALUE10, D_VALUE10, N_VALUE11, C_VALUE11, D_VALUE11, N_VALUE12, C_VALUE12, D_VALUE12, N_VALUE13, C_VALUE13, D_VALUE13, N_VALUE14, C_VALUE14, D_VALUE14, N_VALUE15, C_VALUE15, D_VALUE15, N_VALUE16, C_VALUE16, D_VALUE16, N_VALUE17, C_VALUE17, D_VALUE17, N_VALUE18, C_VALUE18, D_VALUE18, N_VALUE19, C_VALUE19, D_VALUE19, N_VALUE20, C_VALUE20, D_VALUE20, ACTIVE_BEGIN_DATE, ACTIVE_END_DATE, CREATED_BY, CREATION_DATE, LAST_UPDATED_BY, LAST_UPDATE_DATE, LAST_UPDATE_LOGIN  
        ) VALUES (
            :MAP_ID, :MAP_DEFINITION_ID, :N_VALUE1, :C_VALUE1, :D_VALUE1, :N_VALUE2, :C_VALUE2, :D_VALUE2, :N_VALUE3, :C_VALUE3, :D_VALUE3, :N_VALUE4, :C_VALUE4, :D_VALUE4, :N_VALUE5, :C_VALUE5, :D_VALUE5, :N_VALUE6, :C_VALUE6, :D_VALUE6, :N_VALUE7, :C_VALUE7, :D_VALUE7, :N_VALUE8, :C_VALUE8, :D_VALUE8, :N_VALUE9, :C_VALUE9, :D_VALUE9, :N_VALUE10, :C_VALUE10, :D_VALUE10, :N_VALUE11, :C_VALUE11, :D_VALUE11, :N_VALUE12, :C_VALUE12, :D_VALUE12, :N_VALUE13, :C_VALUE13, :D_VALUE13, :N_VALUE14, :C_VALUE14, :D_VALUE14, :N_VALUE15, :C_VALUE15, :D_VALUE15, :N_VALUE16, :C_VALUE16, :D_VALUE16, :N_VALUE17, :C_VALUE17, :D_VALUE17, :N_VALUE18, :C_VALUE18, :D_VALUE18, :N_VALUE19, :C_VALUE19, :D_VALUE19, :N_VALUE20, :C_VALUE20, :D_VALUE20, :ACTIVE_BEGIN_DATE, :ACTIVE_END_DATE, :CREATED_BY, :CREATION_DATE, :LAST_UPDATED_BY, :LAST_UPDATE_DATE, :LAST_UPDATE_LOGIN 
        )"""

target_vm_seq = "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S.NEXTVAL"

# Build the insert script
insert_script = build_insert_script(vm_values, insert_vm, target_vm_seq)

# Print or write the insert script to a file
print(insert_script)


NameError: name 'Timestamp' is not defined